In [ ]:
import jwt
import datetime
from flask import Flask, request, jsonify
import requests
import nest_asyncio  # Fix for running Flask in Jupyter

# Apply nest_asyncio for Jupyter environments
nest_asyncio.apply()

# Secret key for JWT
SECRET_KEY = "your_secret_key"

# Flask app
app = Flask(__name__)

# Simulated database
users = {}  # Stores user information in the format: username -> {password, role}
roles = {
    "Admin": ["read", "write", "delete"],
    "User": ["read"],
    "Moderator": ["read", "write"]
}

# Utility: Generate JWT token
def generate_token(username, role):
    payload = {
        "username": username,
        "role": role,
        "exp": datetime.datetime.utcnow() + datetime.timedelta(hours=1)
    }
    return jwt.encode(payload, SECRET_KEY, algorithm="HS256")

# Route: Home (root)
@app.route("/", methods=["GET"])
def home():
    return jsonify({
        "message": "Welcome to the RBAC System! Use the endpoints /register, /login, and /resource/<action>.",
        "endpoints": {
            "register": "POST /register (JSON: username, password, role)",
            "login": "POST /login (JSON: username, password)",
            "resource": "GET /resource/<action> (Header: Authorization: <token>)"
        }
    })

# Route: Register user
@app.route("/register", methods=["POST"])
def register():
    data = request.json
    username = data.get("username")
    password = data.get("password")
    role = data.get("role")

    if username in users:
        return jsonify({"message": "User already exists!"}), 400
    if role not in roles:
        return jsonify({"message": "Invalid role!"}), 400

    users[username] = {"password": password, "role": role}
    return jsonify({"message": "User registered successfully!"})

# Route: Login
@app.route("/login", methods=["POST"])
def login():
    data = request.json
    username = data.get("username")
    password = data.get("password")

    user = users.get(username)
    if not user or user["password"] != password:
        return jsonify({"message": "Invalid credentials!"}), 401

    token = generate_token(username, user["role"])
    return jsonify({"token": token})

# Middleware: Authenticate token
def authenticate(token):
    try:
        decoded = jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
        return decoded
    except jwt.ExpiredSignatureError:
        return None

# Route: Access resource
@app.route("/resource/<action>", methods=["GET"])
def resource(action):
    token = request.headers.get("Authorization")
    if not token:
        return jsonify({"message": "Token is missing!"}), 401

    decoded = authenticate(token)
    if not decoded:
        return jsonify({"message": "Invalid or expired token!"}), 401

    username = decoded["username"]
    role = decoded["role"]

    if action in roles.get(role, []):
        return jsonify({"message": f"Access granted for {action} action."})
    else:
        return jsonify({"message": "Access denied!"}), 403

# Test Code to interact with the Flask application
def register_user(username, password, role):
    url = f"http://127.0.0.1:5000/register"
    data = {
        "username": username,
        "password": password,
        "role": role
    }
    response = requests.post(url, json=data)
    print("Register User Response:", response.json())

def login_user(username, password):
    url = f"http://127.0.0.1:5000/login"
    data = {
        "username": username,
        "password": password
    }
    response = requests.post(url, json=data)
    if response.status_code == 200:
        print("Login Response:", response.json())
        return response.json().get("token")
    else:
        print("Login failed:", response.json())
        return None

def access_resource(action, token):
    url = f"http://127.0.0.1:5000/resource/{action}"
    headers = {
        "Authorization": token
    }
    response = requests.get(url, headers=headers)
    print(f"Access Resource ({action}) Response:", response.json())

# Run the Flask application and interact with it
if __name__ == "__main__":
    from threading import Thread
    # Run the Flask app in a separate thread
    def run_flask():
        app.run(debug=False, use_reloader=False)

    # Start Flask server in a background thread
    flask_thread = Thread(target=run_flask)
    flask_thread.start()

    # Wait for Flask server to start before proceeding
    import time
    time.sleep(2)

    # Now perform the actions (register, login, and resource access)
    username = "testuser"
    password = "testpass"
    role = "User"  # Change role to Admin, User, or Moderator as needed

    # Step 1: Register the user
    register_user(username, password, role)

    # Step 2: Log in and get JWT token
    token = login_user(username, password)
    
    if token:
        # Step 3: Access a resource (change 'read' to the required action)
        action = "read"  # Change this to 'write', 'delete', etc. depending on the user's role
        access_resource(action, token)